In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt

In [54]:
cost_model_file = '../rust-dp/data/cm.csv'
opt_value_file = '../rust-dp/data/values.csv'
learned_value_file = '../rust-dp/data/values.csv'
max_num_steps = 4
switch_to_update = set([1, 2, 3])

In [55]:
def read_values(file_path):
    values = [[] for _ in range(max_num_steps)]
    switch_idx = [[] for _ in range(max_num_steps)]

    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            val = float(row[0])
            num_steps_left = int(row[1]) - 1  # for idx
            switch = [int(i) for i in row[2:]]
            
            values[num_steps_left].append(val)
            switch_idx[num_steps_left].append(switch)

    return values, switch_idx

In [56]:
def read_cost_model(file_path):
    cost_model = {}  # switch_down (sorted) -> cost
    
    lc = 0
    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if lc == 0:
                lc += 1
                continue
                
            cost = float(row[0])
            switch_down = [int(i) for i in row[1:]]
            
            switch_key = tuple(sorted(switch_down))
            cost_model[switch_key] = cost

    return cost_model

In [58]:
cost_model = read_cost_model(cost_model_file)
opt_values, opt_switch_idx = read_values(opt_value_file)
learned_values, learned_switch_idx = read_values(learned_value_file)

In [65]:
switches_left = set(switch_to_update)
cost_so_far = 0
for i in range(max_num_steps, 0, -1):
    step_idx = i - 1
    opt_i = np.argmin(opt_values[step_idx])
    learned_i = np.argmin(learned_values[step_idx])
    if i == 1:
        print('Step {} left, optimal value: {}, learned value: {}'.format(
              i, opt_values[step_idx][opt_i],
              learned_values[step_idx][learned_i]))
    else:
        # plan for the next step
        l_next_i = np.argmin(learned_values[step_idx - 1])
        switches_down = switches_left - set(
            learned_switch_idx[step_idx - 1][l_next_i])
        
        # get cost 
        cost = cost_model[tuple(sorted(switches_down))]
        cost_so_far += cost
        switches_left -= switches_down
        
        # report
        print(('Step {} left, optimal value: {}, ' + 
               'learned value: {}, cost + next value: {} + {}, ' + 
               'cost so far: {}').format(
              i, opt_values[step_idx][opt_i],
              learned_values[step_idx][learned_i],
              cost, learned_values[step_idx - 1][l_next_i],
              cost_so_far))
        

Step 4 left, optimal value: 0.0, learned value: 0.0, cost + next value: 3.0 + 0.0, cost so far: 3.0
Step 3 left, optimal value: 0.0, learned value: 0.0, cost + next value: 2.0 + 0.0, cost so far: 5.0
Step 2 left, optimal value: 0.0, learned value: 0.0, cost + next value: 1.0 + 0.0, cost so far: 6.0
Step 1 left, optimal value: 0.0, learned value: 0.0
